In [1]:
#%% import packages
import numpy as np
import pandas as pd
import os
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy import stats

#%% define path
data_path = './data/'

#%% import data
df_order = pd.read_csv(data_path + 'df_order.csv')
df_region = pd.read_csv(data_path + 'df_region.csv')
df_completed = pd.read_csv(data_path + 'df_completed.csv')

#%% join tables and preprocess
# add indicator for weekday, where 1 = Sunday and 7 = Saturday
df_order['created_weekday'] = [a%7 for a in df_order['created_day_id']]
df_order.loc[df_order['created_weekday'] == 0,'created_weekday'] = 7

# convert time to timedelta
df_order['created_time'] = pd.to_timedelta(df_order['created_time'])
df_completed['completed_time'] = pd.to_timedelta(df_completed['completed_time'])

# convert created time into hour blocks for analysis later
df_order['created_hour'] = [a.seconds // 3600 for a in df_order['created_time']]

# for ease of operations, convert days and times to a datetime starting from an arbitrary date: 1 Aug 2021 (sunday)
df_order['created_datetime'] = [dt.datetime(year=2021,month=8,day=1) + dt.timedelta(days=a-1) + b \
                                for a,b in zip(df_order['created_day_id'],df_order['created_time'])]
df_completed['completed_datetime'] = [dt.datetime(year=2021,month=8,day=1) + dt.timedelta(days=a-1) + b \
                                for a,b in zip(df_completed['completed_day_id'],df_completed['completed_time'])]

# join orders and completed, leave out regional characteristics for now
df_combined = pd.merge(df_order, df_completed, on = 'order_request_id', how = 'outer')
#df_combined = pd.merge(df_combined, df_region, left_on = 'pickup_location_region', right_on = 'region')
#df_combined = df_combined.drop(columns = 'pickup_location_region')

# add indicator for whether the order was completed
df_combined.loc[~pd.isnull(df_combined['completed_day_id']),'completed_ind'] = 1
df_combined.loc[pd.isnull(df_combined['completed_day_id']),'completed_ind'] = 0

# calculate time to complete order
df_combined['completion_time'] = df_combined['completed_datetime'] - df_combined['created_datetime']

FileNotFoundError: [Errno 2] No such file or directory: './data/df_order.csv'